In [41]:
import pandas as pd
import pickle
from sklearn.preprocessing import MinMaxScaler, StandardScaler

import statsmodels.api as sm
import statsmodels.formula.api as smf
from patsy import dmatrices
from statsmodels.stats.outliers_influence import variance_inflation_factor

import pylab as plt
%matplotlib inline

import seaborn as sns

## SES data

In [2]:
pe = pd.read_csv('PovertyEstimates.csv', thousands=',').rename(columns={'FIPStxt': 'fips'})
pe['fips'] = pe['fips'].apply(lambda x: str(x).zfill(5))
pe = pe.set_index('fips')[['POVALL_2018', 'PCTPOVALL_2018', 'MEDHHINC_2018']]
pe.shape

(3193, 3)

## Rural data

In [3]:
df_rural = pd.read_csv('County_Rural_Lookup.csv', thousands=',', usecols=[0, 7])
df_rural = df_rural.rename(columns={'2015 GEOID': 'fips', '2010 Census \nPercent Rural': 'perc_rural_pop'})
df_rural = df_rural.set_index('fips').dropna()
print(df_rural.shape)

(3142, 1)


## Demographic data

In [4]:
dm_raw = pd.read_csv('cc-est2019-alldata.csv', encoding='ISO-8859-1', dtype={'STATE': str, 'COUNTY': str})
dm_raw = dm_raw[dm_raw.YEAR==12]
dm_raw['fips'] = dm_raw.STATE + dm_raw.COUNTY
dm_raw = dm_raw.set_index('fips')
dm_raw.head()

,SUMLEV,STATE,COUNTY,STNAME,CTYNAME,YEAR,AGEGRP,TOT_POP,TOT_MALE,TOT_FEMALE,...,HWAC_MALE,HWAC_FEMALE,HBAC_MALE,HBAC_FEMALE,HIAC_MALE,HIAC_FEMALE,HAAC_MALE,HAAC_FEMALE,HNAC_MALE,HNAC_FEMALE
fips,,,,,,,,,,,,,,,,,,,,,
01001,50,01,001,Alabama,Autauga County,12,0,55869,27092,28777,...,778,687,89,93,40,27,15,19,16,11
01001,50,01,001,Alabama,Autauga County,12,1,3277,1713,1564,...,76,53,10,6,6,5,3,4,3,3
01001,50,01,001,Alabama,Autauga County,12,2,3465,1787,1678,...,83,59,2,10,8,2,2,0,1,1
01001,50,01,001,Alabama,Autauga County,12,3,3851,1977,1874,...,84,67,11,12,2,2,1,2,2,1
01001,50,01,001,Alabama,Autauga County,12,4,3659,1854,1805,...,55,68,7,6,4,5,0,4,3,0


In [5]:
dm_raw['minority'] = dm_raw.TOT_POP - (dm_raw.WA_MALE + dm_raw.WA_FEMALE)
dm_raw['black'] = dm_raw.BA_MALE + dm_raw.BA_FEMALE
dm_raw['hispanic'] = dm_raw.H_MALE + dm_raw.H_FEMALE
dm_all = dm_raw[dm_raw.AGEGRP == 0][['minority', 'black', 'hispanic', 'TOT_POP']]

In [6]:
dm_old = dm_raw[['AGEGRP', 'TOT_POP']][dm_raw.AGEGRP >= 14].reset_index().groupby('fips').sum()
dm_old = dm_old.rename(columns={'TOT_POP': '65yrs'}).drop(['AGEGRP'], axis=1)

In [7]:
dm = dm_old.join(dm_all)

In [8]:
columns = dm.columns
for c in ['65yrs', 'minority', 'black', 'hispanic']:
    pc = 'perc_' + c
    dm[pc] = dm[c] / dm.TOT_POP


### Sanity check for demographic dataframe (dm)
passed

In [9]:
summation = dm.sum(axis=0)
perc_black = summation['black'] / summation['TOT_POP']
print(f'black percentage = {100 * perc_black:.1f}%')
perc_hispanic = summation['hispanic'] / summation['TOT_POP']
print(f'hispanic percentage = {100 * perc_hispanic:.1f}%')
perc_minority = summation['minority'] / summation['TOT_POP']
print(f'minority percentage = {100 * perc_minority:.1f}%')
print(f"population over 65yrs = {summation['65yrs']}")

black percentage = 13.4%
hispanic percentage = 18.5%
minority percentage = 23.7%
population over 65yrs = 54058263.0


## Covid data

In [33]:
df_covid_raw = pd.read_csv('time_series_covid19_confirmed_US_2020-10-22.csv').dropna()
df_covid_raw['FIPS'] = df_covid_raw['FIPS'].apply(lambda x: str(int(x)).zfill(5))
df_covid_raw = df_covid_raw.rename(columns={'FIPS': 'fips'}).set_index('fips')
df_covid_raw = df_covid_raw.drop([
    'UID', 'iso2', 'iso3', 'code3', 
    'Admin2', 'Province_State', 'Country_Region', 
    'Lat', 'Long_', 'Combined_Key'], axis=1)

df_covid_raw['1/21/20'] = 0

In [44]:
last_date = '10/22/20'
periods = {
    'p1': ['1/21/20', '4/1/20'],
    'p2': ['3/17/20', '5/1/20'],
    'p3': ['4/1/20', '6/8/20'],
    'p4': ['5/1/20', '7/15/20'],
    'p5': ['6/8/20', '8/10/20'],
    'p6': ['7/15/20', '9/10/20'],
    'p7': ['8/10/20', '10/4/20'],
    'p8': ['9/10/20', last_date]
}
with open('periods.pkl', 'wb') as handle:
    pickle.dump(periods, handle)

In [35]:
df_covid_period = pd.DataFrame()
df_covid_period['covid'] = df_covid_raw[last_date]

for key, interval in periods.items():
    a, b = interval
    df_covid_period[key] = df_covid_raw[b] - df_covid_raw[a]
    
df_covid_period.head()
    
# for i in range(len(covid_chosen))[::-1][:-1]:
#     col = covid_chosen[i]
#     col_prev = covid_chosen[i - 1]
#     df_covid_period[col] = df_covid_chosen[col] - df_covid_chosen[col_prev]

# df_covid_period = df_covid_period[covid_chosen]
# df_covid_period['covid'] = df_covid_raw[df_covid_raw.columns[-1]]
# print(df_covid_period.shape)

,covid,p1,p2,p3,p4,p5,p6,p7,p8
fips,,,,,,,,,
01001,2023,8,42,264,701,902,670,654,610
01003,6475,20,174,300,1324,3331,3179,2422,1797
01005,997,0,42,197,384,379,192,345,379
01007,811,3,42,76,196,363,328,244,245
01009,1893,5,39,68,328,739,732,844,794


In [37]:
df_covid = df_covid_period.join(dm[['TOT_POP']], how='inner')
print(df_covid.shape)

for col in df_covid_period.columns:
    new_col = col + '_pc'
    df_covid[new_col] = df_covid[col] / df_covid['TOT_POP']

df_covid.drop(['TOT_POP'], axis=1, inplace=True)
print(df_covid.head())

(3142, 10)
       covid  p1   p2   p3    p4    p5    p6    p7    p8  covid_pc     p1_pc  \
fips                                                                           
01001   2023   8   42  264   701   902   670   654   610  0.036210  0.000143   
01003   6475  20  174  300  1324  3331  3179  2422  1797  0.029005  0.000090   
01005    997   0   42  197   384   379   192   345   379  0.040387  0.000000   
01007    811   3   42   76   196   363   328   244   245  0.036215  0.000134   
01009   1893   5   39   68   328   739   732   844   794  0.032736  0.000086   

          p2_pc     p3_pc     p4_pc     p5_pc     p6_pc     p7_pc     p8_pc  
fips                                                                         
01001  0.000752  0.004725  0.012547  0.016145  0.011992  0.011706  0.010918  
01003  0.000779  0.001344  0.005931  0.014922  0.014241  0.010850  0.008050  
01005  0.001701  0.007980  0.015555  0.015353  0.007778  0.013976  0.015353  
01007  0.001876  0.003394  0.008752  0

## Risk data
For Ishanu:
1. The `A_B` column of the risk dataframe `rf` means llk of disease model `B` generating disease time series `A`;
2. Now risk is defined to be `2 / (llk Staph generating Staph + llk Strep generating Strep)`;
3. Now risk_flu is defined to be `1 / llk of flu model generating flu sequence`;
4. Feel free to try other formulae for risk.

In [38]:
rf = pd.read_csv('county_pop_risk_covid.csv', dtype={'county': str})
rf = rf.rename(columns={'county': 'fips'}).set_index('fips')
rf['risk'] = 2 / (rf['Staphylococcus_Staphylococcus'] + rf['Streptococcal_Streptococcal'])

rf.fillna(rf.mean(), inplace=True)
print(rf.shape)

(3094, 14)


### Combine demographic, SES, rural, risk, and covid dataframe
For Ishanu: df here has a column of population over 65yrs

In [39]:
risk_cols = ['risk', 'risk_flu']
df = dm.join(pe, how='inner')\
    .join(rf[risk_cols], how='inner')\
    .join(df_rural, how='inner')\
    .join(df_covid, how='inner')\
    .rename(columns={
        'POVALL_2018': 'poverty',
        'PCTPOVALL_2018': 'perc_poverty', 
        'MEDHHINC_2018': 'income', 
        'TOT_POP': 'population'})
print(df.columns)

Index(['65yrs', 'minority', 'black', 'hispanic', 'population', 'perc_65yrs',
       'perc_minority', 'perc_black', 'perc_hispanic', 'poverty',
       'perc_poverty', 'income', 'risk', 'risk_flu', 'perc_rural_pop', 'covid',
       'p1', 'p2', 'p3', 'p4', 'p5', 'p6', 'p7', 'p8', 'covid_pc', 'p1_pc',
       'p2_pc', 'p3_pc', 'p4_pc', 'p5_pc', 'p6_pc', 'p7_pc', 'p8_pc'],
      dtype='object')


In [40]:
print(df.shape)
df.to_csv('combined_data.csv', float_format='%.6f')

(3094, 33)
